In [18]:
! pip uninstall webgui_jupyter_widgets -y
! pip install webgui_jupyter_widgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [2]:
tend = 0.1
u0 = exp(-100*( (x-0.5)**2 + (y-0.5)**2))

fes = H1(mesh, order=3)
u,v = fes.TnT()

m = BilinearForm(u*v*dx).Assemble()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
f = LinearForm(fes).Assemble()

In [3]:
tau = 0.005
mstarinv = (m.mat + tau*a.mat).CreateSparseMatrix().Inverse()

In [4]:
gfu = GridFunction(fes)
gfu.Set(u0)

scene = Draw(gfu, deformation=True, euler_angles=[-60,-7,-15])
sleep(3)

for j in range(int(tend/tau)):
    res = f.vec - a.mat * gfu.vec
    w = mstarinv * res
    gfu.vec.data += tau*w
    scene.Redraw()
    sleep(0.2)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

In [5]:
ftot = LinearForm(1*v*dx).Assemble()
print ("total hear energy: ", InnerProduct(ftot.vec, gfu.vec))

print ("gfu(0.5, 0.5) =", gfu(0.5, 0.5))

fpoint = LinearForm(fes)
fpoint += v(0.5, 0.5)
fpoint.Assemble()
print ("point value: ", InnerProduct(fpoint.vec, gfu.vec))

total hear energy:  0.03141635086587979
gfu(0.5, 0.5) = 0.03465200882130126
point value:  0.03465200882130126


In [6]:
energy_values = []
time_values = []
time_current = 0.0
ftot = LinearForm(1*v*dx).Assemble()
energy_initial = InnerProduct(ftot.vec, gfu.vec)
energy_values.append(energy_initial)
time_values.append(time_current)
print("Initial total heat energy: ", energy_initial)
num_steps = int(tend / tau)
for j in range(num_steps):
    res = f.vec - a.mat * gfu.vec
    w = mstarinv * res
    gfu.vec.data += tau * w
    time_current += tau
    current_energy = InnerProduct(ftot.vec, gfu.vec)
    energy_values.append(current_energy)
    time_values.append(time_current)
    scene.Redraw()
    sleep(0.2)

Initial total heat energy:  0.03141635086587979
